In [1]:
from sf_runoff import daily_climatology, spatial_avg_daily_input
from nested_CV import SVR_nested_CV_gridsearch, SVR_PCA_nested_CV_gridsearch

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV,TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


import matplotlib.pyplot as plt

import os

import pdb
import seaborn as sns

path=r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\Documents\conda\daily_input\\'
# ## Feature selection
daily_input = pd.read_csv(path+'Etsch - Branzoll_85550PG_8555QQ0010A_19761231000000_20210721000000.csv', index_col=0, parse_dates=True)

spatial_avg_daily_input

In [ ]:
#define the possible parameters value (where Gridsearch is applied)

C_range=np.logspace(-2, 2, 10)
epsilon_range=np.logspace(-6, -2, 10)
t_range=[12]
n_splits=6
test_size=730


C,eps=SVR_nested_CV_gridsearch(daily_input, C_range,epsilon_range, t_range,n_splits,test_size)
print(f'C:{C}')
print(f'epsilon:{eps}')

Fitting 1 folds for each of 100 candidates, totalling 100 fits
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Fitting 1 folds for each of 100 candidates, totalling 100 fits


In [ ]:
import 
n_range = [17, 50, 200]
C2,eps2,n2=SVR_PCA_nested_CV_gridsearch(daily_input, C_range,n_range, epsilon_range, t_range,n_splits,test_size)
